<a href="https://colab.research.google.com/github/mrdbourke/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07. Milestone Project 1: 🍔👁 Food Vision Big™

Trong notebook trước ([transfer learning phần 3: scaling up](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/06_transfer_learning_in_tensorflow_part_3_scaling_up.ipynb)), chúng ta đã xây dựng Food Vision mini: mô hình transfer learning đánh bại kết quả ban đầu của [tài liệu Food101](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/) chỉ với 10% dữ liệu.

Có thể các bạn đang thắc mắc điều gì sẽ xảy ra nếu chúng ta sử dụng toàn bộ dữ liệu?

Đây là điều mà chúng ta sẽ tìm hiểu trong notebook này!

Chúng ta sẽ xây dựng Food Vision Big™ (Bài toán thực phẩm), sử dụng toàn bộ dữ liệu từ tập dữ liệu Food101.

Có tổng cộng 75,750 ảnh huấn luyện và 25,250 ảnh kiểm tra.

Và đoán xem...

Lần này **chúng ta đặt mục tiêu vượt qua [DeepFood](https://www.researchgate.net/publication/304163308_DeepFood_Deep_Learning-Based_Food_Image_Recognition_for_Computer-Aided_Dietary_Assessment)**, tài liệu xuất bản năm 2016 sử dụng CNN được huấn luyện 2-3 ngày, đạt được top-1 accuracy (77.4%).

> 🔑 **Lưu ý:** **Top-1 accuracy** nghĩa là "độ chính xác cho đầu ra giá trị kích hoạt softmax cao nhất theo mô hình" (do softmax xuất ra giá trị cho mỗi lớp nhưng top-1 nghĩa là chỉ giá trị lớn nhất mới được đánh giá). **Top-5 accuracy** nghĩa là "độ chính xác cho 5 đầu ra giá trị kích hoạt softmax hàng đầu theo mô hình", hay nói cách khác, true label có xuất hiện trong top 5 giá trị kích hoạt không? Top-5 accuracy score thường cao hơn hẳn so với top-1.

|  | 🍔👁 Food Vision Big™ | 🍔👁 Food Vision mini |
|-----|-----|-----|
| Nguồn dataset | TensorFlow Datasets | Download đã tiền xử lý từ Kaggle |
| Dữ liệu huấn luyện | 75,750 ảnh | 7,575 ảnh |
| Dữ liệu kiểm tra | 25,250 ảnh | 25,250 ảnh |
| Mixed precision | Có | Không |
| Data loading | Thực hiện với tf.data API | Hàm đã xây của TensorFlow |  
| Kết quả mục tiêu | 77.4% top-1 accuracy (vượt [tài liệu DeepFood](https://arxiv.org/abs/1606.05675)) | 50.76% top-1 accuracy (vượt [tài liệu Food101](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/static/bossard_eccv14_food-101.pdf)) |

*Bảng so sánh sự khác nhau giữa Food Vision Big (notebook này) với Food Vision mini (notebook trước).*

Cùng với nỗ lực vượt qua kết quả của tài liệu DeepFood, chúng ta sẽ tìm hiểu hai phương pháp giúp cải thiện đáng kể tốc độ huấn luyện mô hình:
1. Prefetching
2. Mixed precision training

Chúng ta sẽ tìm hiểu thêm về chúng sau.

## Những điều chúng ta sẽ tìm hiểu

* Sử dụng TensorFlow Datasets để download và khám phá dữ liệu
* Tạo hàm tiền xử lý dữ liệu
* Batch và chuẩn bị tập dữ liệu để lập mô hình (**giúp tập dữ liệu chạy nhanh hơn**)
* Tạo callback lập mô hình
* Thiết lập **mixed precision training**
* Xây dựng mô hình feature extraction (xem [transfer learning phần 1: feature extraction](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/04_transfer_learning_in_tensorflow_part_1_feature_extraction.ipynb))
* Tinh chỉnh mô hình feature extraction (xem [transfer learning phần 2: fine-tuning](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/05_transfer_learning_in_tensorflow_part_2_fine_tuning.ipynb))
* Xem các kết quả huấn luyện trên TensorBoard

## Cách sử dụng notebook này

Các bạn có thể đọc qua các mô tả và code (tất cả sẽ chạy, trừ những cell mắc lỗi có chủ đích), nhưng có một lựa chọn tốt hơn.

Tự viết toàn bộ code.

Nghiêm túc đấy. Hãy tạo notebook mới và tự viết lại từng dòng. Kiểm tra xem bạn có thể thay đổi nó không và lý do cho điều đó.

Bạn không cần viết mô tả bằng văn bản nhưng tự viết lại code là một cách tuyệt vời để có trải nghiệm thực tiễn.

Đừng lo lắng nếu mắc sai sót, ai cũng đều mắc lỗi cả. Cách thực hiện tốt hơn và mắc ít lỗi hơn là **viết nhiều code hơn**.

> 📖 **Tài liệu:** Xem bộ tài liệu khóa học đầy đủ trên GitHub: https://github.com/mrdbourke/tensorflow-deep-learning

## Kiểm tra GPU

Với notebook này, chúng ta sẽ thực hiện khác biệt.

Chúng ta sẽ sử dụng mixed precision training.

được đề xuất trong [TensorFlow 2.4.0](https://blog.tensorflow.org/2020/12/whats-new-in-tensorflow-24.html) (đặc trưng rất mới tại thời điểm viết).

**Mixed precision training** thực hiện những gì?

Mixed precision training sử dụng tổ hợp của các kiểu dữ liệu precision đơn lẻ (float32) và half-preicison (float16) để tăng tốc độ huấn luyện mô hình (gấp 3 lần trên các GPU hiện đại).

Chúng ta sẽ nói về điều này sau, hãy đọc [tài liệu của TensorFlow về mixed precision](https://www.tensorflow.org/guide/mixed_precision) để biết thêm chi tiết.

Còn bây giờ, trước khi đi tiếp, nếu muốn sử dụng mixed precision training, chúng ta cần đảm bảo GPU cấp nguồn cho phiên bản Google Colab của chúng ta (nếu bạn đang sử dụng Google Colab) tương thích.

Để mixed precision training hoạt động, cần truy cập vào GPU với hệ số tương thích tính toán trên 7.0.

Google Colab cung cấp các GPU P100, K80 và T4, tuy nhiên, **GPU P100 và K80 không tương thích với mixed precision training**.

Do đó, trước khi tiến hành, chúng ta cần đảm bảo **có quyền truy cập vào GPU Tesla T4 ở phiên bản Google Colab**.

Nếu bạn đang không dùng Google Colab, có thể tìm danh sách các [khả năng tính toán khác nhau của Nvidia GPU trên website Nvidia developer](https://developer.nvidia.com/cuda-gpus#compute).

> 🔑 **Lưu ý:** Nếu bạn chạy cell bên dưới và thấy P100 hoặc K80, hãy thử vào Runtime -> Factory Reset Runtime (lưu ý: điều này sẽ loại bất cứ dữ liệu và biến đã lưu nào khỏi phiên bản Colab) rồi thử lại để lấy T4.

In [4]:
# Nếu không dùng Google Colab, điều này sẽ xuất ra "Tesla T4", nếu không,
# bạn sẽ không thể sử dụng mixed precision training
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


Do mixed precision training được đề xuất trong TensorFlow 2.4.0 nên hãy đảm bảo bạn sử dụng TensorFlow ít nhất là từ bản 2.4.0.

In [5]:
# Kiểm tra phiên bản TensorFlow (nên là từ 2.4.0 trở lên)
import tensorflow as tf
print(tf.__version__)

2.15.0


## Tạo hàm hỗ trợ

Chúng ta đã tạo một chuỗi các hàm hỗ trợ ở notebook trước. Thay vì viết lại (điều này rất dài dòng), chúng ta sẽ import file [`helper_functions.py`](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/helper_functions.py) từ kho GitHub.

In [6]:
# Lấy file hàm hỗ trợ
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-04-21 19:01:38--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-04-21 19:01:38 (30.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [7]:
# Import chuỗi các hàm hỗ trợ cho notebook (chúng ta đã tạo/sử dụng trong notebook trước)
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Sử dụng TensorFlow Datasets để download dữ liệu

Ở notebook trước, chúng ta đã download ảnh thực phẩm (từ [tập dữ liệu Food101](https://www.kaggle.com/dansbecker/food-101/home)) trên Google Storage.

Đây là quy trình điển hình mà chúng ta sẽ dùng nếu đang làm việc với tập dữ liệu của mình.

Tuy nhiên, có một cách khác để chuẩn bị sẵn các tập dữ liệu dùng với TensorFlow.

Chúng ta có thể truy cập nhiều tập dữ liệu phổ biến nhất trong thế giới Machine learning (thường đề cập tới và sử dụng như các đánh giá xếp hạng) qua [TensorFlow Datasets (TFDS)](https://www.tensorflow.org/datasets/overview).

**TensorFlow Datasets** là gì?

Nơi dành cho cho các tập dữ liệu ML đã chuẩn bị và sẵn sàng để sử dụng.

Tại sao cần sử dụng TensorFlow Datasets?

* Load dữ liệu đã có trong Tensor
* Luyện tập trên các tập dữ liệu đã thiết lập tốt
* Thử nghiệm với các kỹ thuật data loading khác nhau (như chúng ta sẽ dùng trong notebook này
* Thử nghiệm với các đặc trưng TensorFlow mới một cách nhanh chóng (như mixed precision training)

Tại sao không sử dụng TensorFlow Datasets?

* Các tập dữ liệu tĩnh (chúng không thay đổi, giống như các tập dữ liệu trong thực tế)
* Có thể không thích hợp với bài toán vấn đề của bạn (nhưng tốt để thử nghiệm)

Để bắt đầu, chúng ta sẽ import TensorFlow Datasets dưới alias `tfds`.


In [8]:
# Lấy TensorFlow Datasets
import tensorflow_datasets as tfds

Để tìm tất cả các tập dữ liệu có sẵn trong TensorFlow Datasets, bạn có thể dùng phương thức `list_builders()`.

Sau đó, chúng ta có thể kiểm tra xem tập dữ liệu mà chúng ta theo dõi (`"food101"`) có hiện diện không.

In [9]:
# Liệt kê các tập dữ liệu có sẵn
datasets_list = tfds.list_builders() # lấy tất cả các tập dữ liệu có sẵn trong TFDS
print("food101" in datasets_list) # tập dữ liệu mà chúng ta đang tìm có sẵn không?

True


Có vẻ tập dữ liệu mà chúng ta theo dõi đã có sẵn (lưu ý có nhiều tập dữ liệu có sẵn nhưng chúng ta chỉ theo dõi Food101).

Chúng ta có thể sử dụng phương thức [`tfds.load()`](https://www.tensorflow.org/datasets/api_docs/python/tfds/load) để truy cập vào tập dữ liệu Food101 từ TFDS.

Cụ thể, chúng ta sẽ phải truyền cho nó một ít tham số để cho nó biết chúng ta đang tìm gì:
* `name` (str) : tập dữ liệu mục tiêu (chẳng hạn: `"food101"`)
* `split` (list, tùy chọn) : phần dữ liệu chia tách của tập dữ liệu mà chúng ta đang theo dõi (chẳng hạn: `["train", "validation"]`)
  * tham số `split` khá phức tạp. Xem [tài liệu](https://github.com/tensorflow/datasets/blob/master/docs/splits.md) để biết thêm.
* `shuffle_files` (bool) : có xáo trộn các file trong download hay không, mặc định là `False`
* `as_supervised` (bool) : `True` để download các mẫu dữ liệu ở định dạng tuple (`(data, label)`) hoặc `False` cho định dạng dictionary
* `with_info` (bool) : `True` để download tập dữ liệu metadata (nhãn, số mẫu,...)

> 🔑 **Lưu ý:** Việc gọi phương thức `tfds.load()` sẽ bắt đầu download một tập dữ liệu mục tiêu vào ổ đĩa nếu tham số `download=True` được thiết lập (mặc định). Tập dữ liệu này có thể trên 100GB nên hãy đảm bảo bạn có đủ dung lượng.

In [ ]:
# Load dữ liệu (mất khoảng 5-6 phút trong Google Colab)
(train_data, test_data), ds_info = tfds.load(name="food101", # tập dữ liệu mục tiêu để lấy từ TFDS
                                             split=["train", "validation"], # chúng ta nên lấy phần chia tách dữ liệu nào? lưu ý: không phải tất cả các tập dữ liệu đều có train, valid, test
                                             shuffle_files=True, # có xáo trộn file trong download không?
                                             as_supervised=True, # download dữ liệu ở định dạng tuple (sample, label), chẳng hạn (image, label)
                                             with_info=True) # có gồm tập dữ liệu metadata không? nếu có, tfds.load() trả về tuple (data, ds_info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Sau vài phút download, chúng ta đã truy cập vào toàn bộ tập dữ liệu Food101 (ở dạng tensor), sẵn sàng để lập mô hình.

Giờ hãy lấy một ít thông tin từ tập dữ liệu của chúng ta, bắt đầu với tên lớp.

 Việc lấy tên lớp từ tập dữ liệu TFDS yêu cầu download biến "`dataset_info`" (bằng cách dùng tham số `as_supervised=True` trong phương thức `tfds.load()`, **lưu ý:** điều này sẽ chỉ hoạt động với các tập dữ liệu được giám sát trong TFDS)

Chúng ta có thể truy cập tên lớp của một tập dữ liệu cụ thể với thuộc tính `dataset_info.features` và truy cập thuộc tính `names` của khóa `"label"`.

In [ ]:
# Các đặc trưng của Food101 TFDS
ds_info.features

In [ ]:
# Lấy tên lớp
class_names = ds_info.features["label"].names
class_names[:10]

### Khám phá dữ liệu Food101 từ TFDS

Chúng ta đã download tập dữ liệu Food101 từ TFDS, hãy thực hiện những gì mà một nhà khám phá dữ liệu tốt nên làm.

Nói cách khác, hãy "visualize, visualize, visualize (trực quan hóa)"

Tìm hiểu một vài chi tiết về tập dữ liệu của chúng ta:
* Shape của dữ liệu đầu vào (image tensor)
* Kiểu dữ liệu của dữ liệu đầu vào
* Label của dữ liệu đầu vào trông như thế nào? (chẳng hạn: mã hóa one-hot với mà hóa nhãn)
* Các nhãn có khớp với tên lớp không?

Để thực hiện, hãy lấy một mẫu từ dữ liệu huấn luyện (sử dụng [phương thức `.take()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take)) và khám phá nó.

In [ ]:
# Lấy một mẫu từ dữ liệu huấn luyện
train_one_sample = train_data.take(1) # các mẫu ở định dạng (image_tensor, label)

Vì chúng ta đã sử dụng tham số `as_supervised=True` trong phương thức `tfds.load()` ở trên nên các mẫu dữ liệu có cấu trúc dạng tuple `(data, label)` hoặc trong trường hợp của chúng ta `(image_tensor, label)`.

In [ ]:
# Một mẫu của dữ liệu huấn luyện trông như thế nào?
train_one_sample

Hãy lặp qua một mẫu huấn luyện đơn lẻ và lấy một số thông tin từ `image_tensor` và `label`.

In [ ]:
# Xuất ra thông tin về mẫu huấn luyện
for image, label in train_one_sample:
  print(f"""
  Image shape: {image.shape}
  Image dtype: {image.dtype}
  Target class from Food101 (tensor form): {label}
  Class name (str form): {class_names[label.numpy()]}
        """)

Bởi vì chúng ta sẽ thiết lập tham số `shuffle_files=True` trong phương thức `tfds.load()` ở trên nên việc chạy cell trên một vài lần sẽ cho các kết quả khác nhau mỗi lần.

Kiểm tra chúng, bạn sẽ nhận thấy một số ảnh có các shape khác nhau, chẳng hạn: `(512, 342, 3)` và `(512, 512, 3)` (height, width, color_channels).

Hãy xem một trong số các image tensor của tập dữ liệu Food101 từ TFDS trông như thế nào.

In [ ]:
# Image tensor từ tập dữ liệu Food101 của TFDS trông như thế nào?
image_1 = image_1
image

In [ ]:
# Giá trị lớn nhất và nhỏ nhất là bao nhiêu?
tf.reduce_min(image), tf.reduce_max(image)

Có vẻ image tensor có các giá trị trong khoảng 0-255 (các giá trị màu red, green, blue tiêu chuẩn) và các giá trị thuộc kiểu dữ liệu `unit8`.

Chúng ta có thể sẽ phải tiền xử lý chúng trước khi truyền chúng vào mạng nơ-ron. Tuy nhiên, chúng ta sẽ giải quyết điều này sau.

Trong lúc đó, hãy xem chúng ta có thể vẽ mẫu ảnh không.

### Vẽ ảnh từ TensorFlow Datasets

Chúng ta đã thấy các image tensor ở định dạng tensor, bây giờ hãy thực hiện theo phương châm của chúng ta.

"Visualize, visualize, visualize!"

Hãy vẽ một trong các mẫu ảnh sử dụng [`matplotlib.pyplot.imshow()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html) và đặt title thành tên lớp mục tiêu.

In [ ]:
# Vẽ image tensor
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()]) # thêm title vào ảnh bằng cách lập chỉ mục danh sách class_names
plt.axis(False);

Tuyệt!

Có vẻ dữ liệu Food101 mà chúng ta thu được từ TFDS tương tự với những tập dữ liệu mà chúng ta đang sử dụng ở các notebook trước.

Bây giờ hãy tiền xử lý nó và chuẩn bị để sử dụng với mạng nơ-ron.

## Tạo các hàm tiền xử lý dữ liệu

Ở các notebook trước, chúng ta đã dùng phương thức [`tf.keras.preprocessing.image_dataset_from_directory()`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory) để load ảnh vào khi chúng ở định dạng folder.

Thực hiện như vậy nghĩa là dữ liệu được load vào một định dạng sẵn sàng để dùng cho mô hình.

Tuy nhiên, do chúng ta đã download dữ liệu từ TensorFlow Datasets nên cần thực hiện nhiều bước tiền xử lý trước khi sẵn sàng lập mô hình.

 Cụ thể, dữ liệu của chúng ta hiện:

* Ở kiểu dữ liệu `uint8`
* Gồm tất cả các tensor có kích thước khác nhau (các ảnh có kích thước khác nhau)
* Không biến đổi tỷ lệ (giá trị điểm ảnh trong khoảng 0-255)

Trong khi đó, các mô hình muốn dữ liệu:

* Ở kiểu dữ liệu `float32`
* Có tất cả các tensor có kích thước như nhau (các batch yêu cầu toàn bộ tensor có shape giống nhau, chẳng hạn: `(224, 224, 3)`)  
* Có biến đổi tỷ lệ (các giá trị trong khoảng 0-1), còn được gọi là chuẩn hóa

Để xử lý những điều này, chúng ta sẽ tạo hàm `preprocess_img()`:

* Resize tensor ảnh đầu vào thành kích thước chỉ định với [`tf.image.resize()`](https://www.tensorflow.org/api_docs/python/tf/image/resize)
* Chuyển kiểu dữ liệu hiện tại của tensor ảnh đầu vào thành `tf.float32` với [`tf.cast()`](https://www.tensorflow.org/api_docs/python/tf/cast)

> 🔑 **Lưu ý:** Mô hình EfficientNetBX đã huấn luyện trước trong [`tf.keras.applications.efficientnet`](https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet) (thứ chúng ta sẽ sử dụng) có phép tái tỷ lệ tích hợp. Nhưng với nhiều kiến trúc mô hình khác, bạn sẽ muốn tái tỷ lệ dữ liệu của mình (chẳng hạn: lấy các giá trị trong khoảng 0-1). Điều này có thể được tích hợp bên trong hàm "`preprocess_img()`" (như bên dưới) hoặc trong mô hình ở dạng [`tf.keras.layers.experimental.preprocessing.Rescaling`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling) layer.

In [ ]:
# Tạo hàm để tiền xử lý ảnh
def preprocess_img(image, label, img_shape=224):
  """
  Chuyển kiểu dữ liệu ảnh từ 'uint8' -> 'float32' và reshape ảnh thành
  [img_shape, img_shape, color_channels]
  """
  image = tf.image.resize(image, [img_shape, img_shape]) # reshape thành img_shape
  return tf.cast(image, tf.float32), label # trả về tuple (float32_image, label)

Hàm `preprocess_img()` ở trên lấy ảnh và nhãn làm đầu vào (dù nó không ảnh hưởng gì tới nhãn) vì tập dữ liệu của chúng ta hiện đang ở cấu trúc tuple `(image, label)`.

Hãy thử hàm trên một ảnh mục tiêu.

In [ ]:
# Tiền xử lý một mẫu ảnh đơn lẻ và kiểm tra đầu ra
preprocessed_img = preprocess_img(image, label)[0]
print(f"Image before preprocessing:\n {image[:2]}...,\nShape: {image.shape},\nDatatype: {image.dtype}\n")
print(f"Image after preprocessing:\n {preprocessed_img[:2]}...,\nShape: {preprocessed_img.shape},\nDatatype: {preprocessed_img.dtype}")

Có vẻ hàm `preprocess_img()` đang hoạt động như mong đợi.

Ảnh đầu vào được chuyển từ `uint8` sang `float32` và reshape từ shape hiện tại thành `(224, 224, 3)`.

Nó trông như thế nào?

In [ ]:
# Chúng ta vẫn có thể ve ảnh đã tiền xử lý miễn là
# chia cho 255 (để tương thích vói matplotlib)
plt.imshow(preprocessed_img/255.)
plt.title(class_names[label])
plt.axis(False);

Hình thức ăn này khiến tôi đói rồi. Sao chúng ta không bắt đầu mô hình hóa nó?

## Batch & chuẩn bị tập dữ liệu

Trước khi mô hình hóa dữ liệu, chúng ta cần biến nó thành các batch.

Tại sao?

Vì việc tính toán trên các batch sẽ có tiết kiệm bộ nhớ hơn.

Chúng ta biến dữ liệu từ 101,000 image tensor và nhãn (kết hợp huấn luyện và kiểm tra) thành các batch gồm 32 cặp ảnh và nhãn, cho phép khớp với bộ nhớ của CPU.

Để thực hiện một cách hiệu quả, chúng ta sẽ tận dụng nhiều phương thức từ [`tf.data` API](https://www.tensorflow.org/api_docs/python/tf/data).

> 📖 **Tài liệu:** Hãy xem tài liệu của TensorFlow [Better performance with the tf.data API](https://www.tensorflow.org/guide/data_performance) để load dữ liệu theo cách hiệu quả nhất có thể.

Cụ thể, chúng ta sẽ sử dụng:

* [`map()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) - ánh xạ một hàm đã xác định trước tới tập dữ liệu mục tiêu (chẳng hạn: `preprocess_img()` tới image tensor)
* [`shuffle()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle) - xáo trộn ngẫu nhiên các phần tử của tập dữ liệu mục tiêu trên `buffer_size` (lý tưởng là `buffer_size` bằng với kích thước của tập dữ liệu; tuy nhiên, điều này có thể ảnh hưởng tới bộ nhớ
* [`batch()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch) - biến các phần tử của tập dữ liệu mục tiêu thành các batch (do tham số `batch_size` xác định kích thước)
* [`prefetch()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch) - chuẩn bị các batch dữ liệu tiếp theo trong khi các batch dữ liệu khác được tính toán (cải thiện tốc độ load dữ liệu nhưng tốn bộ nhớ)
* Extra: [`cache()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache) - trữ (lưu chúng để sử dụng sau) các phần tử trong tập dữ liệu mục tiêu, lưu thời lượng load (chỉ hoạt động nếu tập dữ liệu nhỏ vừa khớp với bộ nhớ, các phiên bản Colab tiêu chuẩn chỉ có bộ nhớ 12GB)

Những điều cần lưu ý:
- Không thể nhóm hàng loạt các tensort có shape khác nhau (ví dụ: kích thước ảnh khác nhau cần reshape ảnh trước nên dùng hàm `preprocess_img()`)
- `shuffle()` giữ buffer (vùng đệm) của số mà bạn truyền cho nó các ảnh đã xáo trộn, con số này sẽ là tất cả các mẫu trong tập huấn luyện, tuy nhiên nếu tập huấn luyện lớn thì buffer này sẽ không khớp với bộ nhớ (con số lớn như 1000 hoặc 10000 thường đã đủ để xáo trộn)
- Đối với các phương thức với tham số `num_parallel_calls` có sẵn (chẳng hạn như `map()`), đặt nó thành `num_parallel_calls=tf.data.AUTOTUNE` sẽ đồng thời tiền xử lý và cải thiện đáng kể tốc độ
- Không thể sử dụng `cache()` trừ khi tập dữ liệu có thể vừa với bộ nhớ

Phía trên có khá nhiều thứ. Nhưng sau khi đã viết code ở dưới, mọi thứ sẽ bắt đầu dễ hiểu hơn.

Chúng ta sẽ đi qua mọi thứ theo trình tự sau:

```
Dataset ban đầu (ví dụ: train_data) -> map() -> shuffle() -> batch() -> prefetch() -> PrefetchDataset
```

Điều này tương đương với diễn giải sau:

> "Hãy ánh xạ hàm tiền xử lý này trên tập dữ liệu huấn luyện, sau đó xáo trộn các phần từ trước khi nhóm chúng lại với nhau và đảm bảo bạn chuẩn bị các batch mới (prefetch) trong khi mô hình đang xem qua batch hiện tại."

![](https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/07-prefetching-from-hands-on-ml.png)

*Những gì xảy ra khi dùng prefetching (nhanh hơn) với không dùng prefetching (chậm hơn). **Nguồn:** [Aurélien Géron. "Hands-On Machine Learning with Scikit-Learn", Keras & TensorFlow, trang 422](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/).*  


In [ ]:
# Ánh xạ hàm tiền xử lý tới dữ liệu huấn luyện (và thực hiện song song)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Xáo trộn train_data và biến nó thành các batch và prefetch nó (load nhanh hơn)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Ánh xạ hàm tiền xử lý tới dữ liệu kiểm tra
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Biến dữ liệu kiểm tra thành các batch (không cần xáo trộn)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

Giờ hãy kiểm tra xem các tập dữ liệu đã chuẩn bị trông như thế nào.

In [ ]:
train_data, test_data

Xuất sắc! Có vẻ dữ liệu hiện đang ở dạng tuple `(image, label)` với kiểu dữ liệu `(tf.float32, tf.int64)`, đây là những gì mà mô hình theo đuổi.

> 🔑 **Lưu ý:** Chúng ta có thể thoát mà không cần gọi phương thức `prefetch()` ở cuối tập dữ liệu, tuy nhiên các bạn có thể sẽ thấy tốc độ load dữ liệu sẽ chậm hơn đáng kể khi xây dựng mô hình. Do đó phần lớp pipeline đầu vào tập dữ liệu sẽ kết thúc với một lệnh gọi [`prefecth()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch).


## Tạo modelling callback

Vì chúng ta sẽ huấn luyện trên một lượng lớn dữ liệu và quá trình này có thể tốn nhiều thời gian nên hãy thiết lập một số modelling callback để chắc chắn nhật ký huấn luyện của mô hình được theo dõi và mô hình được checkpoint (lưu lại) sau nhiều mốc huấn luyện khác nhau.

Để thực hiện những điều này, chúng ta sẽ sử dụng các callback sau:
* [`tf.keras.callbacks.TensorBoard()`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) - cho phép theo dõi lịch sử huấn luyện mô hình để chúng ta có thể kiểm tra nó sau (**lưu ý:** chúng ta đã tạo callback này trước khi import nó từ `helper_functions.py` làm `create_tensorboard_callback()`)
* [`tf.keras.callbacks.ModelCheckpoint()`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) - lưu tiến trình mô hình ở nhiều khoảng do đó chúng ta có thể load và sử dụng lại nó sau mà không cần huấn luyện lại
  * Việc checkpoint cũng hữu ích nên chúng ta có thể bắt đầu tinh chỉnh mô hình ở một epoch nhất định và hoàn nguyên về trạng thái trước đó nếu việc tinh chỉnh không mang lại lợi ích gì

In [ ]:
# Tạo TensorBoard callback (đã có "create_tensorboard_callback()" từ notebook trước)
from helper_functions import create_tensorboard_callback

# Tạo ModelCheckpoint callback để lưu tiến trình mô hình
checkpoint_path = "model_checkpoints/cp.ckpt" # việc lưu trọng số yêu cầu ".ckpt" extension
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      montior="val_acc", # lưu các trọng số của mô hình với validation accuracy tốt nhất
                                                      save_best_only=True, # chỉ lưu những trọng số tốt nhất
                                                      save_weights_only=True, # chỉ lưu các trọng số của mô hình (không phải toàn bộ mô hình)
                                                      verbose=0) # không in ra dù mô hình có đang được lưu hay không

## Thiết lập mixed precision training

Chúng ta đã đề cập tới mixed precision training ở trên.

Tuy nhiên, chúng ta đã không giải thích nhiều về điều này.

Tensor trong TensorFlow thường mặc định có kiểu dữ liệu float32 (trừ những loại được chỉ định).

Trong khoa học máy tính, float32 còn được gọi là [định dạng single-precision floating-point](https://en.wikipedia.org/wiki/Single-precision_floating-point_format). 32 nghĩa là nó thường chiếm 32 bit trong bộ nhớ máy tính.

Bộ nhớ của GPU có hạn, do đó nó chỉ có thể xử lý một vài float32 cùng một lúc.

Lúc này mixed precision training có tác dụng.

Mixed precision training dùng kết hợp tensor float16 và float32 để sử dụng bộ nhớ GPU tốt hơn.

Bạn có đoán được float16 nghĩa là gì không?

Nếu float32 là single-precision floating-point thì float16 là [định dạng half-precision floating-point](https://en.wikipedia.org/wiki/Half-precision_floating-point_format). Nếu bạn đoán vậy thì đúng rồi đấy! Còn nếu không đoán đúng thì cũng không sao, giờ bạn đã biết.

Với những tensor ở định dạng float16, mỗi phần tử chiếm 16 bit trong bộ nhớ máy tính.

Vậy điều này có gì hữu ích?

Như đã đề cập trước đó, khi sử dụng mixed precision training, mô hình sẽ tận dụng các kiểu dữ liệu float32 và float32 để dùng ít bộ nhớ hơn khi có thể và chạy nhanh hơn (dùng ít bộ nhớ hơn mỗi tensor nghĩa là có thể tính toán đồng thời nhiều tensor).

Như vậy, sử dụng mixed precision training có thể cải thiện hiệu suất của các GPU hiện đại (GPU với hệ số khả năng tính toán trên 7.8) lên gấp 3 lần.

Hãy đọc qua [hướng dẫn của TensorFlow về mixed precision](https://www.tensorflow.org/guide/mixed_precision) để xem thêm các diễn giải chi tiết (ít nhất cũng hãy xem quan phần tổng kết).

![](https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/07-mixed-precision-speedup-equals-3x-gpu.png)
*Do mixed precision training sử dụng kết hợp các kiểu dữ liệu float32 và float16 nên các bạn sẽ thấy tốc độ của các GPU hiện đại tăng lên gấp 3.*

> 🔑 **Lưu ý:** Nếu GPU không có hệ số trên 7.0 (chẳng hạn: P100 trong Colab) thì mixed precision sẽ không hoạt động (xem ["Supported Hardware"](https://www.tensorflow.org/guide/mixed_precision#supported_hardware) trong hướng dẫn về mixed precision để biết thêm chi tiết).

> 📖 **Tài liệu:** Để tìm hiểu thêm về precision trong khoa học máy tính (chi tiết đại lượng số mà máy tính biểu thị), hãy xem trên [Wikipedia](https://en.wikipedia.org/wiki/Precision_(computer_science)) (và các tài liệu đi kèm).

Hãy xem làm thế nào để khởi động mixed precision training trong TensorFlow.

Điều tuyệt vời là [`tensorflow.keras.mixed_precision`](https://www.tensorflow.org/api_docs/python/tf/keras/mixed_precision/) API giúp chúng ta dễ dàng bắt đầu.

Trước tiên, chúng ta sẽ import API rồi sử dụng phương thức [`set_global_policy()`](https://www.tensorflow.org/api_docs/python/tf/keras/mixed_precision/set_global_policy) để đặt *dtype policy* thành `"mixed_float16"`.


In [ ]:
# Khởi động mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16") # đạt global policy thành mixed precision

Cell trên sẽ chạy mà không bị lỗi miễn là GPU có khả năng tính toán trên 7.0.

Chúng ta có thể kiểm tra global dtype policy (là policy sẽ được các layer dùng trong mô hình) bằng phương thức [`mixed_precision.global_policy()`](https://www.tensorflow.org/api_docs/python/tf/keras/mixed_precision/global_policy).

In [ ]:
mixed_precision.global_policy() # nên xuất ra "mixed_float16"

Vì global dtype policy hiện giờ là `"mixed_float16"` nên mô hình sẽ từ động tận dụng các biến float16 khi có thể, do đó tăng tốc huấn luyện.

## Xây dựng mô hình feature extraction

Callback: sẵn sàng triển khai.

Mixed precision: đã khởi động.

Hãy xây dựng mô hình.

Do tập dữ liệu khá lớn nên chúng ta sẽ tinh chỉnh pretrained model đang có (EfficienetNetB0).

Nhưng trước khi tiến hành fine-tuning, hãy thiết lập mô hình feature-extraction.

Xin nhắc lại trình tự điển hình cho transfer learning như sau:

1. Xây dựng mô hình feature extraction (thay vài layer trên đầu của pretrained model)
2. Huấn luyện vài epoch với các layer thấp hơn bị đóng băng
3. Tinh chỉnh nếu cần với nhiều layer không bị đóng băng

![](https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/07-feature-extraction-then-fine-tune.png)
*Trước khi tinh chỉnh, tốt nhất là huấn luyện mô hình feature extraction với các layer trên cùng tùy chỉnh.*

Để xây dựng mô hình feature extraction (đề cập trong [Transfer Learning trong TensorFlow Phần 1: Feature extraction](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/04_transfer_learning_in_tensorflow_part_1_feature_extraction.ipynb)), chúng ta sẽ:
* Sử dụng `EfficientNetB0` từ [`tf.keras.applications`](https://www.tensorflow.org/api_docs/python/tf/keras/applications) đã huấn luyện trên ImageNet làm base model
  * Chúng ta sẽ download nó mà không cần các layer trên cùng bằng tham số `include_top=False`, do đó chúng ta có thể tạo các layer đầu ra của riêng mình
* Đóng băng các layer của base model để có thể sử dụng các pattern đã tìm hiểu trước mà mô hình tìm thấy trên ImageNet
* Kết hợp đầu vào, base model, pooling layer và các layer đầu ra trong [Functional model](https://keras.io/guides/functional_api/)
* Biên dịch mô hình Functional sử dụng thuật toán tối ưu Adam và [sparse categorical crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy) làm hàm mất mát (do các nhãn **không** được mã hóa one-hot)
* Khớp mô hình cho 3 epoch bằng TensorBoard và ModelCheckpoint callback

> 🔑 **Lưu ý:** Vì chúng ta đang sử dụng mixed precision training nên mô hình cần một layer đầu ra riêng biệt với hard-coded `dtype=float32`, ví dụ: `layers.Activation("softmax", dtype=tf.float32)`. Điều này đảm bảo đầu ra của mô hình được trả về kiểu dữ liệu float32, ổn định hơn về mặt số so với kiểu dữ liệu float16 (quan trọng với tính toán loss). Xem phần ["Building the model"](https://www.tensorflow.org/guide/mixed_precision#building_the_model) trong hướng dẫn của TensorFlow để biết thêm chi tiết.

![](https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/07-mixed-precision-code-before-and-after.png)
*Khởi động mixed precision trong TensorFlow với 3 dòng code.*

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Tạo base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False # freeze base model layers đóng băng các layer của base model

# Tạo Functional model
inputs = layers.Input(shape=input_shape, name="input_layer")
# Lưu ý: mô hình EfficientNetBX có tích hợp rescaling nhưng nếu mô hình của bạn không có, bạn có thể có layer như dưới đây
# x = preprocessing.Rescaling(1./255)(x)
x = base_model(inputs, training=False) #  chỉ đặt base_model thành chế độ suy luận
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(class_names))(x) # muốn một nơ-ron đầu ra mỗi lớp
# Tách activation của layer đầu ra, do đó chúng ta có thể xuất ra float32 activation
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)

# Biên dịch mô hình
model.compile(loss="sparse_categorical_crossentropy", # Sử dụng sparse_categorical_crossentropy khi các nhãn *không phải* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
# Kiểm tra mô hình
model.summary()

## Kiểm tra các dtype policy của layer (chúng ta có đang dùng mixed precision không?)

Mô hình đã sẵn sàng!

Trước đó chúng ta có nói rằng mixed precision API sẽ tự động thay đổi dtype policy của các layer thành bất kỳ global dtype policy nào (trong trường hợp này nó là `"mixed_float16"`).

Chúng ta có thể kiểm tra điều này bằng cách lặp qua các layer của mô hình và in ra các thuộc tính của layer như `dtype` và `dtype_policy`.

In [ ]:
# Kiểm tra thuộc tính dtype_policy của các layer trong mô hình
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy) # Kiểm tra dtype policy của các layer

Qua phần trên, chúng ta thấy:
* `layer.name` (str) : tên của layer mà chúng ta đọc được, được xác định bởi tham số `name` khi xây dựng
* `layer.trainable` (bool) : liệu một layer có thể huấn luyện hay không (tất cả các layer đều có thể huấn luyện trừ layer efficientnetb0 vì chúng ta đã đặt thuộc tính `trainable` thành `False`
* `layer.dtype` : kiểu dữ liệu mà layer lưu trữ các biến của nó trong đó
* `layer.dtype_policy` : kiểu dữ liệu mà một layer tính toán trong đó

> 🔑 **Lưu ý:** Một layer có thể có dtype `float32` và dtype policy `"mixed_float16"` vì nó lưu trữ các biến (trọng số và độ chệch) trong `float32` (ổn định hơn về mặt số học), tuy nhiên nó tính toán trong `float16` (nhanh hơn).

Chúng ta cũng có thể kiểm tra các chi tiết tương tự cho base model.


In [28]:
# Kiểm tra các layer trong base model và xem chúng đang sử dụng dtype policy nào
for layer in model.layers[1].layers[:20]: # chỉ kiểm tra 20 layer đầu tiên để tiết kiệm không gian đầu ra
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 False float32 <Policy "float32">
rescaling False float32 <Policy "mixed_float16">
normalization False float32 <Policy "mixed_float16">
rescaling_1 False float32 <Policy "mixed_float16">
stem_conv_pad False float32 <Policy "mixed_float16">
stem_conv False float32 <Policy "mixed_float16">
stem_bn False float32 <Policy "mixed_float16">
stem_activation False float32 <Policy "mixed_float16">
block1a_dwconv False float32 <Policy "mixed_float16">
block1a_bn False float32 <Policy "mixed_float16">
block1a_activation False float32 <Policy "mixed_float16">
block1a_se_squeeze False float32 <Policy "mixed_float16">
block1a_se_reshape False float32 <Policy "mixed_float16">
block1a_se_reduce False float32 <Policy "mixed_float16">
block1a_se_expand False float32 <Policy "mixed_float16">
block1a_se_excite False float32 <Policy "mixed_float16">
block1a_project_conv False float32 <Policy "mixed_float16">
block1a_project_bn False float32 <Policy "mixed_float16">
block2a_expand_conv False float32 <

> 🔑 **Lưu ý:** Mixed precision API tự động khiến các layer có lợi từ việc sử dụng `"mixed_float16"` dtype policy dùng nó. Nó cũng ngăn cản các layer không dùng nó khỏi việc sử dụng nó (chẳng hạn: layer chuẩn hóa ở đầu base model).

## Khớp mô hình feature extraction

Mô hình đã trông khá tốt. Hãy khớp nó vớ dữ liệu.

3 epoch đã đủ để các layer trên cùng điều chỉnh trọng số của chúng với dữ liệu ảnh thức ăn.

Để tiết kiệm thời gian mỗi epoch, chúng ta sẽ chỉ kiểm định trên 15% dữ liệu kiểm tra.

In [29]:
# Khớp mô hình với các callback
history_101_food_classes_feature_extract = model.fit(train_data,
                                                     epochs=3,
                                                     steps_per_epoch=len(train_data),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),
                                                     callbacks=[create_tensorboard_callback("training_logs",
                                                                                            "efficientnetb0_101_classes_all_data_feature_extract"),
                                                                model_checkpoint])

Saving TensorBoard log files to: training_logs/efficientnetb0_101_classes_all_data_feature_extract/20240421-174226
Epoch 1/3
2368/2368 [==============================] - 245s 93ms/step - loss: 1.7197 - accuracy: 0.5813 - val_loss: 1.1303 - val_accuracy: 0.7015
Epoch 2/3
2368/2368 [==============================] - 205s 85ms/step - loss: 1.2002 - accuracy: 0.6898 - val_loss: 1.0275 - val_accuracy: 0.7211
Epoch 3/3
2368/2368 [==============================] - 208s 87ms/step - loss: 1.0550 - accuracy: 0.7244 - val_loss: 0.9978 - val_accuracy: 0.7248


Có vẻ mô hình feature extraction đang thực hiện khá tốt. Chúng ta có nên đánh giá trên toàn bộ tập dữ liệu kiểm tra không?

In [30]:
# Đánh giá mô hình (phiên bản chưa lưu) trên toàn bộ tập dữ liệu kiểm tra
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

790/790 [==============================] - 58s 73ms/step - loss: 0.9983 - accuracy: 0.7286


[0.9983130693435669, 0.7285940647125244]

Do chúng ta đã sử dụng `ModelCheckpoint` callback nên chúng ta có một phiên bản đã lưu của mô hình trong directory `model_checkpoints`.

Hãy load nó vào và đảm bảo nó hoạt động tốt.

## Load và đánh giá các trọng số checkpoint

Chúng ta có thể load và đánh giá các checkpoint của mô hình bằng cách:

1. Clone (Sao chép) mô hình sử dụng [`tf.keras.models.clone_model()`](https://www.tensorflow.org/api_docs/python/tf/keras/models/clone_model) để tạo bản sao của mô hình feature extraction có các tham số được thiết lập lại.
2. Gọi phương thức `load_weights()` trong mô hình đã clone, truyền cho nó đường dẫn tới nơi lưu các trọng số đã chekpoint.
3. Gọi `evaluate()` trên mô hình đã clone với các trọng số đã load.

Nhắc lại rằng các checkpoint rất hữu ích khi chúng ta thực hiện thử nghiệm như tinh chỉnh mô hình. Trong trường hợp tinh chỉnh mô hình feature extraction mà không thấy có cải thiện thì chúng ta có thể hoàn nguyên về phiên bản mô hình đã checkpoint

In [31]:
# Clone mô hình mà chúng ta đã tạo (điều này sẽ thiết lập lại các trọng số)
cloned_model = tf.keras.models.clone_model(model)
cloned_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 pooling_layer (GlobalAvera  (None, 1280)              0         
 gePooling2D)                                                    
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                                 
 softmax_float32 (Activatio  (None, 101)               0         
 n)                                                              
                                                             

In [32]:
# Các checkpoint được lưu ở đâu?
checkpoint_path

'model_checkpoints/cp.ckpt'

In [33]:
# Load các trọng số được checkpoint vào mô hình đã clone
cloned_model.load_weights(checkpoint_path)

Mỗi lần thay đổi mô hình (gồm việc load các trọng số), chúng ta phải biên dịch lại.

In [34]:
# Biên dịch cloned_model (với các tham số tương tự như mô hình ban đầu)
cloned_model.compile(loss="sparse_categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [35]:
# Đánh giá mô hình đã clone với các trọng số đã load (nên có hệ số giống với mô hình đã huấn luyện)
results_cloned_model_with_loaded_weights = cloned_model.evaluate(test_data)

790/790 [==============================] - 87s 103ms/step - loss: 1.3641 - accuracy: 0.6364


Kết quả của mô hình đã clone với trọng số đã load phải rất gần với kết quả của mô hình feature extraction (nếu cell dưới đây lỗi thì sẽ xảy ra lỗi).

Clone mô hình sẽ bảo toàn `dtype_policy` của các layer (nhưng không bảo toàn trọng số) nên chúng ta có thể tiếp tục tinh chỉnh với mô hình đã clone nếu muốn và nó vẫn sẽ sử dụng mixed precision dtype policy.

In [49]:
# Kiểm tra các layer trong base model và xem chúng đang sử dụng dtype policy nào
for layer in cloned_model.layers[1].layers[:20]: # chỉ kiểm tra 20 layer đầu tiên để tiết kiệm dung lượng
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 True float32 <Policy "float32">
rescaling False float32 <Policy "mixed_float16">
normalization False float32 <Policy "mixed_float16">
rescaling_1 False float32 <Policy "mixed_float16">
stem_conv_pad False float32 <Policy "mixed_float16">
stem_conv False float32 <Policy "mixed_float16">
stem_bn False float32 <Policy "mixed_float16">
stem_activation False float32 <Policy "mixed_float16">
block1a_dwconv False float32 <Policy "mixed_float16">
block1a_bn False float32 <Policy "mixed_float16">
block1a_activation False float32 <Policy "mixed_float16">
block1a_se_squeeze False float32 <Policy "mixed_float16">
block1a_se_reshape False float32 <Policy "mixed_float16">
block1a_se_reduce False float32 <Policy "mixed_float16">
block1a_se_expand False float32 <Policy "mixed_float16">
block1a_se_excite False float32 <Policy "mixed_float16">
block1a_project_conv False float32 <Policy "mixed_float16">
block1a_project_bn False float32 <Policy "mixed_float16">
block2a_expand_conv False float32 <P

## Lưu toàn bộ mô hình vào file

Chúng ta cũng có thể lưu toàn bộ mô hình bằng phương thức [`save()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save).

Vì mô hình khá lớn, chúng ta có thể sẽ cần lưu nó vào Google Drive (nếu đang sử dụng Google Colab) để load nó cho các sử dụng sau này.

> 🔑 **Lưu ý:** Lưu ý: Việc lưu vào Google Drive yêu cầu phải cài Google Drive (vào Files -> Mount Drive).

In [50]:
# ## Lưu mô hình vào Google Drive (tùy chọn)

# # Tạo save path vào drive
# save_dir = "drive/MyDrive/tensorflow_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision/"
# # os.makedirs(save_dir) # Tạo directory nếu nó không tồn tại

# # Lưu mô hình
# model.save(save_dir)

Chúng ta cũng có thể lưu nó trực tiếp vào Google Colab instance của chúng ta.

> 🔑 **Lưu ý:** Lưu ý: Bộ nhớ Google Colab là tạm thời và mô hình sẽ tự xóa (cùng với bất kỳ file đã lưu nào khác) khi phiên Colab hết hạn.

In [51]:
# Lưu mô hình cục bộ (nếu đang sử dụng Google Colab, mô hình đã lưu sẽ kết thúc Colab instance)
save_dir = "07_efficientnetb0_feature_extract_model_mixed_precision"
model.save(save_dir)

Một lần nữa, chúng ta có thể kiểm tra xem mô hình có được lưu chính xác hay không bằng cách load vào và đánh giá nó.

In [52]:
# Load mô hình đã lưu trước đó ở trên
loaded_saved_model = tf.keras.models.load_model(save_dir)

Việc load `SavedModel` cũng giữ lại tất cả các layer bên dưới `dtype_policy` (chúng ta muốn chúng là `"mixed_float16"`).

In [53]:
# Kiểm tra các layer trong base model và xem chúng đang sử dụng dtype policy nào
for layer in loaded_saved_model.layers[1].layers[:20]: # chỉ kiểm tra 20 layer đầu tiên để tiết kiệm không gian đầu ra
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 True float32 <Policy "float32">
rescaling False float32 <Policy "mixed_float16">
normalization False float32 <Policy "mixed_float16">
rescaling_1 False float32 <Policy "mixed_float16">
stem_conv_pad False float32 <Policy "mixed_float16">
stem_conv False float32 <Policy "mixed_float16">
stem_bn False float32 <Policy "mixed_float16">
stem_activation False float32 <Policy "mixed_float16">
block1a_dwconv False float32 <Policy "mixed_float16">
block1a_bn False float32 <Policy "mixed_float16">
block1a_activation False float32 <Policy "mixed_float16">
block1a_se_squeeze False float32 <Policy "mixed_float16">
block1a_se_reshape False float32 <Policy "mixed_float16">
block1a_se_reduce False float32 <Policy "mixed_float16">
block1a_se_expand False float32 <Policy "mixed_float16">
block1a_se_excite False float32 <Policy "mixed_float16">
block1a_project_conv False float32 <Policy "mixed_float16">
block1a_project_bn False float32 <Policy "mixed_float16">
block2a_expand_conv False float32 <P

In [54]:
# Kiểm tra chất lượng của mô hình đã load (phải giống như results_feature_extract_model)
results_loaded_saved_model = loaded_saved_model.evaluate(test_data)
results_loaded_saved_model

790/790 [==============================] - 63s 77ms/step - loss: 0.9983 - accuracy: 0.7286


[0.9983136057853699, 0.7285940647125244]

In [55]:
# Kết quả của mô hình đã load phải bằng (hoặc ít nhất là rất gần) với kết quả của mô hình trước khi lưu
# Lưu ý: điều này sẽ chỉ hoạt động nếu bạn đã cài đặt các biến kết quả
import numpy as np
assert np.isclose(results_feature_extract_model, results_loaded_saved_model).all()

Đó là những gì chúng ta muốn! Mô hình đã load hoạt động như thường.

> 🔑 **Lưu ý:** Hãy dành một chút thời gian để đảm bảo mô hình được lưu chính xác vì việc huấn luyện trên nhiều dữ liệu có thể tốn thời gian, cho nên đảm bảo là chúng ta không phải huấn luyện liên tục từ đầu.

## Chuẩn bị các layer của mô hình cho fine-tuning

Mô hình feature-extraction đang cho thấy một số kết quả hứa hẹn sau ba epoch. Nhưng do chúng ta có quá nhiều dữ liệu, nên có lẽ chúng ta sẽ phải xem xét những kết quả mà chúng ta có thể nhận được khi fine-tuning (fine-tuning thường hoạt động tốt nhất khi có nhiều dữ liệu).

Các bạn còn nhớ mục tiêu đánh bại [DeepFood paper](https://arxiv.org/pdf/1606.05675.pdf) không?

Họ đạt được độ chính xác 77.4% - top 1 trên Food101 trong vòng 2-3 ngày huấn luyện.

Bạn có nghĩ rằng fine-tuning sẽ giúp chúng ta đạt được điều đó không?

Hãy cùng tìm hiểu.

Trước tiên, hãy load mô hình đã lưu.

> 🔑 **Lưu ý:** Cần nhớ quy trình fine-tuning truyền thống: đóng băng pre-trained base model, sau đó chỉ huấn luyện các layer đầu ra trong một vài vòng lặp để cập nhật trọng số trực tiếp với dữ liệu tùy chỉnh (feature extraction). Tiếp đó sẽ bỏ đóng băng một số hoặc tất cả các layer trong base model và tiếp tục huấn luyện cho đến khi mô hình ngừng cải thiện.

Giống như tất cả các chương trình nấu, tôi đã lưu một mô hình mà tôi đã chuẩn bị trước đó (mô hình feature extraction ở trên) vào Google Storage.

Chúng ta có thể download nó để đảm bảo từ giờ chúng ta sẽ sử dụng cùng một mô hình.

In [56]:
# Download mô hình đã lưu từ Google Storage
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip

--2024-04-21 18:21:24--  https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 142.251.2.207, 2607:f8b0:4023:c0d::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16976857 (16M) [application/zip]
Saving to: ‘07_efficientnetb0_feature_extract_model_mixed_precision.zip’

07_efficientnetb0_f 100%[===================>]  16.19M  82.7MB/s    in 0.2s    

2024-04-21 18:21:24 (82.7 MB/s) - ‘07_efficientnetb0_feature_extract_model_mixed_precision.zip’ saved [16976857/16976857]



In [57]:
# Giải nén SavedModel đã download từ Google Storage
!mkdir downloaded_gs_model # tạo dir mới để lưu trữ mô hình feature extraction đã download
!unzip 07_efficientnetb0_feature_extract_model_mixed_precision.zip -d downloaded_gs_model

Archive:  07_efficientnetb0_feature_extract_model_mixed_precision.zip
   creating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/
   creating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/variables/
  inflating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/variables/variables.data-00000-of-00001  
  inflating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/variables/variables.index  
  inflating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/saved_model.pb  
   creating: downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision/assets/


In [58]:
# Load và đánh giá mô hình GS đã download
loaded_gs_model = tf.keras.models.load_model("/content/downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision")

In [59]:
# Lấy summary của mô hình đã download
loaded_gs_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 pooling_layer (GlobalAvera  (None, 1280)              0         
 gePooling2D)                                                    
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                                 
 softmax_float32 (Activatio  (None, 101)               0         
 n)                                                              
                                                             

Bây giờ hãy đảm bảo rằng mô hình đã load của chúng ta đang hoạt động như mong đợi.

In [60]:
# How does the loaded model perform?
results_loaded_gs_model = loaded_gs_model.evaluate(test_data)
results_loaded_gs_model

790/790 [==============================] - 93s 109ms/step - loss: 1.0881 - accuracy: 0.7066


[1.0880988836288452, 0.7066138386726379]

Tuyệt vời, mô hình đã load của chúng ta đang hoạt động như mong muốn.

Lần đầu tạo mô hình, chúng ta đóng băng tất cả các layer trong base model bằng cách đặt `base_model.trainable=False` nhưng vì chúng ta đã load mô hình từ file, nên hãy kiểm tra xem các layer có thể huấn luyện được không.

In [61]:
# Có bất kỳ layer nào bị đóng băng trong mô hình của chúng ta không?
for layer in loaded_gs_model.layers:
  layer.trainable = True # đặt tất cả các layer thành trainable
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy) # đảm bảo mô hình đã load đang sử dụng mixed precision dtype_policy ("mixed_float16")

input_layer True float32 <Policy "float32">
efficientnetb0 True float32 <Policy "mixed_float16">
pooling_layer True float32 <Policy "mixed_float16">
dense True float32 <Policy "mixed_float16">
softmax_float32 True float32 <Policy "float32">


Có vẻ như mỗi layer trong mô hình đã load đều có thể huấn luyện được. Nhưng điều gì sẽ xảy ra nếu chúng ta tìm hiểu sâu hơn một chút và kiểm tra từng layer trong base model?

> 🤔 **Câu hỏi:** *Layer nào trong mô hình đã load là base model của chúng ta?*

Trước khi lưu Functional model vào file, chúng ta tạo mô hình đó với năm layer (các layer bên dưới được lập chỉ mục 0):
0. Input layer
1. Ppre-trained base model layer (`tf.keras.applications.EfficientNetB0`)
2. Pooling layer
3. Fully-connected (dense) layer
4. Output softmax activation (với float32 dtype)

Do đó, để kiểm tra layer của base model, chúng ta có thể truy cập thuộc tính `layers` của layer ở chỉ mục 1 trong mô hình.

In [62]:
# Kiểm tra các layer trong base model và xem chúng đang sử dụng dtype policy nào
for layer in loaded_gs_model.layers[1].layers[:20]:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 True float32 <Policy "float32">
rescaling True float32 <Policy "mixed_float16">
normalization True float32 <Policy "float32">
stem_conv_pad True float32 <Policy "mixed_float16">
stem_conv True float32 <Policy "mixed_float16">
stem_bn True float32 <Policy "mixed_float16">
stem_activation True float32 <Policy "mixed_float16">
block1a_dwconv True float32 <Policy "mixed_float16">
block1a_bn True float32 <Policy "mixed_float16">
block1a_activation True float32 <Policy "mixed_float16">
block1a_se_squeeze True float32 <Policy "mixed_float16">
block1a_se_reshape True float32 <Policy "mixed_float16">
block1a_se_reduce True float32 <Policy "mixed_float16">
block1a_se_expand True float32 <Policy "mixed_float16">
block1a_se_excite True float32 <Policy "mixed_float16">
block1a_project_conv True float32 <Policy "mixed_float16">
block1a_project_bn True float32 <Policy "mixed_float16">
block2a_expand_conv True float32 <Policy "mixed_float16">
block2a_expand_bn True float32 <Policy "mixed_float

Tuyệt vời, có vẻ như mỗi layer trong base model đều có thể huấn luyện được (không đóng băng) và mọi layer sẽ sử dụng dtype policy `"mixed_policy16"` đều đang sử dụng dtype policy này.

Do chúng ta có rất nhiều dữ liệu (750 hình ảnh x 101 lớp huấn luyện = 75750 hình ảnh huấn luyện), nên hãy giữ tất cả các layer của base model không bị đóng băng.

> 🔑 **Lưu ý:** Nếu có ít dữ liệu (ít hơn 100 hình ảnh mỗi lớp), chúng ta sẽ chỉ cần giải nén và tinh chỉnh ít layer trong base model. Nếu không, chúng ta có nguy cơ gặp overfitting.

## Callback

Chúng ta sắp bắt đầu tinh chỉnh mô hình học sâu với hơn 200 layer, sử dụng hơn 100,000 hình ảnh (75k+ huấn luyện, 25K+ kiểm tra), nghĩa là thời gian huấn luyện mô hình có thể sẽ lâu hơn nhiều so với trước đây.

> 🤔 **Câu hỏi:** *Quá trình huấn luyện sẽ mất bao lâu?*

Có thể là một vài giờ hoặc trong trường hợp [DeepFood paper](https://arxiv.org/pdf/1606.05675.pdf) (baseline mà chúng ta đang cố gắng đánh bại), mô hình hoạt động tốt nhất của họ tốn 2-3 ngày gian huấn luyện.

Chúng ta sẽ chỉ biết mất bao lâu khi bắt đầu huấn luyện.

> 🤔 **Câu hỏi:** *Khi nào thì ngừng huấn luyện?*

Lý tưởng nhất là khi mô hình ngừng cải thiện. Nhưng do bản chất của DL, khó có thể biết chính xác khi nào mô hình sẽ ngừng cải thiện.

May thay, có một giải pháp: [`EarlyStopping` callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping).

`EarlyStopping` callback giám sát một phép đo chất lượng cụ thể cho mô hình(ví dụ: `val_loss`) và khi nó ngừng cải thiện trong một số epoch, thì huấn luyện sẽ tự động dừng lại.

`EarlyStopping` callback kết hợp với `ModelCheckpoint` callback sẽ tự động lưu mô hình thực hiện tốt nhất, chúng ta có thể tiếp tục huấn luyện mô hình không giới hạn số lượng epoch cho đến khi nó ngừng cải thiện.

Hãy thiết lập cả hai callback để theo dõi `val_loss` của mô hình.

In [63]:
# Thiết lập EarlyStopping callback để ngừng huấn luyện nếu val_loss của mô hình không cải thiện trong 3 epoch
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # theo dõi phép đo val loss
                                                  patience=3) # nếu val loss giảm trong 3 epoch liên tiếp, hãy ngừng huấn luyện

# Tạo ModelCheckpoint callback để lưu mô hình tốt nhất trong quá trình fine-tuning
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

Các callback cho fine-tuning đã sẵn sàng.

Nếu bạn định huấn luyện các mô hình lớn thì `ModelCheckpoint` và `EarlyStopping là hai callback mà bạn cần tìm hiểu.

Chúng ta gần như đã sẵn sàng để bắt đầu tinh chỉnh mô hình, tuy nhiên còn một callback nữa mà chúng ta sẽ triển khai, đó là [`ReduceLROnPlateau`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau).

Các bạn còn nhớ learning rate là siêu tham số mô hình quan trọng nhất có thể điều chỉnh không? (nếu không, hãy coi đây như một lời nhắc nhở).

`ReduceLROnPlateau` callback giúp điều chỉnh learning rate.

Giống như `ModelCheckpoint` và `EarlyStopping` callback, `ReduceLROnPlateau` callback theo dõi một phép đo cụ thể và khi phép đo đó ngừng cải thiện, nó sẽ giảm learning rate theo một hệ số nhất định (ví dụ: chia learning rate cho 10).

> 🤔 **Câu hỏi:** *Tại sao cần giảm learning rate?*

Giả sử có một đồng xu ở phía sau ghế sofa và bạn đang cố gắng lấy nó bằng các ngón tay của mình.

Bây giờ, hãy coi learning rate là kích thước của các chuyển động mà bàn tay của bạn thực hiện đối với đồng xu.

Bạn càng đến gần thì khả năng chuyển động tay càng nhỏ, nếu không, đồng xu sẽ bị mất.

Chất lượng lý tưởng của mô hình tương đương với việc lấy đồng xu. Vì vậy, khi quá trình huấn luyện diễn ra và mô hình ngày càng tiến gần hơn đến chất lượng lý tưởng (còn gọi là **hội tụ**), chúng ta muốn số lượng nó học được ngày càng ít đi.

Để thực hiện điều này, chúng ta sẽ tạo một instance của `ReduceLROnPlateau` callback để theo dõi validation loss như `EarlyStopping` callback.

Khi validation loss ngừng cải thiện trong hai hoặc nhiều epoch, chúng ta sẽ giảm learning rate xuống theo hệ số 5 (ví dụ:`0.001` thành `0.0002`).

Và để đảm bảo learning rate không quá thấp (có thể khiến mô hình không học được gì), chúng ta sẽ đặt learning rate tối thiểu thành `1e-7`.

In [64]:
# Tạo learning rate để giảm callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2, # nhân learning rate với 0.2 (giảm 5 lần)
                                                 patience=2,
                                                 verbose=1, # in ra khi nào learning rate giảm
                                                 min_lr=1e-7)

Learning rate đã được giảm!

Trước khi bắt đầu huấn luyện, chúng ta phải biên dịch lại mô hình của mình.

Chúng ta sẽ sử categorical crossentropy làm loss và vì chúng ta đang tinh chỉnh, hãy sử dụng learning rate thấp hơn 10 lần so với mặc định của Adam optimizer (`1e-4` thay vì `1e-3`).

In [65]:
# Biên dịch mô hình
loaded_gs_model.compile(loss="sparse_categorical_crossentropy", # sparse_categorical_crossentropy cho các nhãn *không phải* one-hot
                        optimizer=tf.keras.optimizers.Adam(0.0001), # learning rate thấp hơn 10 lần so với mặc định
                        metrics=["accuracy"])

Mô hình đã được biên dịch.

Bây giờ hãy khớp nó với toàn bộ dữ liệu.

Chúng ta sẽ thiết lập nó để chạy cho 100 epoch.

Do chúng ta sẽ sử dụng `EarlyStopping` callback nên nó có thể dừng lại trước khi đạt 100 epoch.

> 🔑 **Lưu ý:** Chạy cell bên dưới sẽ thiết lập mô hình để tinh chỉnh tất cả các trọng số đã huấn luyện trước trong base model trên tất cả dữ liệu của Food101. Thực hiện như vậy với data pipeline **không được tối ưu hóa** và **không có** mixed precision training sẽ tốn khá nhiều thời gian cho mỗi epoch, tùy thuộc vào loại GPU chúng ta đang dùng (khoảng 15-20 phút đối với Colab GPU). Nhưng đừng lo lắng, code mà chúng ta đã viết ở trên sẽ đảm bảo nó chạy nhanh hơn nhiều (khoảng 4-5 phút mỗi epoch).

In [ ]:
# Bắt đầu tinh chỉnh (tất cả các layer)
history_101_food_classes_all_data_fine_tune = loaded_gs_model.fit(train_data,
                                                        epochs=100, # tinh chỉnh tối đa 100 epoch
                                                        steps_per_epoch=len(train_data),
                                                        validation_data=test_data,
                                                        validation_steps=int(0.15 * len(test_data)), # validation trong quá trình huấn luyện trên 15% dữ liệu kiểm tra
                                                        callbacks=[create_tensorboard_callback("training_logs", "efficientb0_101_classes_all_data_fine_tuning"), # theo dõi nhật ký huấn luyện của mô hình
                                                                   model_checkpoint, # chỉ lưu mô hình tốt nhất trong quá trình huấn luyện
                                                                   early_stopping, # ngừng mô hình sau X epoch mà không có cải thiện
                                                                   reduce_lr]) # giảm learning rate sau X epoch mà không có cải thiện

Saving TensorBoard log files to: training_logs/efficientb0_101_classes_all_data_fine_tuning/20240421-182428
Epoch 1/100
2368/2368 [==============================] - 547s 202ms/step - loss: 0.9207 - accuracy: 0.7514 - val_loss: 0.8057 - val_accuracy: 0.7778 - lr: 1.0000e-04
Epoch 2/100
2368/2368 [==============================] - 465s 194ms/step - loss: 0.5770 - accuracy: 0.8397 - val_loss: 0.7701 - val_accuracy: 0.7903 - lr: 1.0000e-04
Epoch 3/100
1906/2368 [=======================>......] - ETA: 1:14 - loss: 0.3294 - accuracy: 0.9063

> 🔑 **Lưu ý:** Nếu không sử dụng mixed precision hoặc các kỹ thuật như [`prefetch()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch) trong phần *Batch & chuẩn bị tập dữ liệu* thì việc tinh chỉnh mô hình có thể lâu hơn 2.5-3 lần mỗi epoch (xem ví dụ output bên dưới ).

| | Prefetch và mixed precision | Không có prefetch và không có mixed precision |
|-----|-----|-----|
| Thời gian mỗi epoch | ~280-300s | ~1127-1397s |

*Kết quả từ việc tinh chỉnh 🍔👁 Food Vision Big™ trên tập dữ liệu Food101, sử dụng chủ yếu là EfficienetNetB0 với Google Colab Tesla T4 GPU.*

```
Saving TensorBoard log files to: training_logs/efficientB0_101_classes_all_data_fine_tuning/20200928-013008
Epoch 1/100
2368/2368 [==============================] - 1397s 590ms/step - loss: 1.2068 - accuracy: 0.6820 - val_loss: 1.1623 - val_accuracy: 0.6894
Epoch 2/100
2368/2368 [==============================] - 1193s 504ms/step - loss: 0.9459 - accuracy: 0.7444 - val_loss: 1.1549 - val_accuracy: 0.6872
Epoch 3/100
2368/2368 [==============================] - 1143s 482ms/step - loss: 0.7848 - accuracy: 0.7838 - val_loss: 1.0402 - val_accuracy: 0.7142
Epoch 4/100
2368/2368 [==============================] - 1127s 476ms/step - loss: 0.6599 - accuracy: 0.8149 - val_loss: 0.9599 - val_accuracy: 0.7373
```
*Ví dụ về thời gian fine-tuning cho dữ liệu non-prefetched cũng như non-mixed precision training (lâu hơn ~ 2.5-3 lần mỗi epoch).*

Hãy đảm bảo rằng chúng mô hình đã được lưu trước khi chúng ta bắt đầu đánh giá nó.

In [ ]:
# # Lưu mô hình vào Google Drive (tùy chọn)
# loaded_gs_model.save("/content/drive/MyDrive/tensorflow_course/food_vision/07_efficientnetb0_fine_tuned_101_classes_mixed_precision/")

In [ ]:
# Lưu mô hình cục bộ (lưu ý: nếu đang sử dụng Google Colab và lưu mô hình cục bộ, mô hình sẽ bị xóa khi phiên Google Colab kết thúc)
loaded_gs_model.save("07_efficientnetb0_fine_tuned_101_classes_mixed_precision")

Có vẻ như mô hình đã đạt được một số điểm chất lượng từ fine-tuning, hãy đánh giá trên toàn bộ tập dữ liệu kiểm tra và xem liệu có thể đánh bại kết quả của [DeepFood paper](https://arxiv.org/abs/1606.05675) là độ chính xác 77.4% hay không.

In [ ]:
# Đánh giá mô hình đã load được huấn luyện với mixed precision
results_loaded_gs_model_fine_tuned = loaded_gs_model.evaluate(test_data)
results_loaded_gs_model_fine_tuned

Có vẻ như mô hình của chúng ta đã đánh bại các kết quả được đề cập trong DeepFood paper cho Food101 (độ chính xác 77.4% - top 1 của DeepFood so với độ chính xác ~79% top 1 của chúng ta).

## Download mô hình đã tinh chỉnh từ Google Storage

Như đã đề cập trước đó, việc huấn luyện có thể tốn khá nhiều thời gian.

Như bất kỳ chương trình nấu ăn nào, đây là thứ chúng ta chuẩn bị trước ...

Đây là một mô hình được tinh chỉnh, giống hệt như mô hình mà chúng ta đã huấn luyện ở trên nhưng được lưu vào Google Storage để có thể truy cập, nhập và đánh giá.

In [ ]:
# Download và đánh giá mô hình đã tinh chỉnh từ Google Storage
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_fine_tuned_101_classes_mixed_precision.zip

 Mô hình đã download có định dạng zip (`.zip`), vì vậy hãy giải nén nó thành Google Colab instance.

In [ ]:
# Giải nén mô hình đã tinh chỉnh
!mkdir downloaded_fine_tuned_gs_model # tạo directory riêng cho mô hình đã tinh chỉnh được download từ Google Storage
!unzip /content/07_efficientnetb0_fine_tuned_101_classes_mixed_precision -d downloaded_fine_tuned_gs_model

Bây giờ chúng ta có thể load nó bằng phương thức [`tf.keras.models.load_model()`](https://www.tensorflow.org/tutorials/keras/save_and_load) và nhận được summary (nó phải giống hệt như mô hình chúng ta đã tạo ở trên).


In [ ]:
# Load mô hình đã tinh chỉnh từ Google Storage và đánh giá
loaded_fine_tuned_gs_model = tf.keras.models.load_model("/content/downloaded_fine_tuned_gs_model/07_efficientnetb0_fine_tuned_101_classes_mixed_precision")

In [ ]:
# Nhận model summary (có cùng kiến trúc mô hình như trên)
loaded_fine_tuned_gs_model.summary()

Cuối cùng, chúng ta có thể đánh giá mô hình trên dữ liệu kiểm tra (cần load biến `test_data`).

In [ ]:
# Lưu ý: Ngay cả khi bạn đang load mô hình từ Google Storage, bạn vẫn cần load biến test_data để cell này hoạt động
results_downloaded_fine_tuned_gs_model = loaded_fine_tuned_gs_model.evaluate(test_data)
results_downloaded_fine_tuned_gs_model

Mô hình đã lưu của chúng ta đang hoạt động như mong đợi (kết quả tốt hơn so với DeepFood!).

Chúc mừng! Bạn vừa huấn luyện một mô hình thị giác máy tính với chất lượng có tính cạnh tranh với một tài liệu nghiên cứu trong thời gian ngắn hơn rất nhiều (mô hình của chúng ta mất ~20 phút để huấn luyện so với mô hình của DeepFood huấn luyện trong 2-3 ngày)

Nói cách khác, bạn đã cải tiến Food Vision!

Nếu bạn muốn nâng cao hơn, có thể thử sử dụng mô hình [`EfficientNetB4`](https://www.tensorflow.org/api_docs/python/tf/keras/applications/EfficientNetB4) (phiên bản lớn hơn của `EfficientNetB0`). Ở thời điểm viết bài, họ EfficientNet có [kết quả phân loại tân tiến nhất](https://paperswithcode.com/sota/fine-grained-image-classification-on-food-101) trên tập dữ liệu Food101.

> 📖 **Tài liệu:** Để biết mô hình nào hiện đang hoạt động tốt nhất trên một tập dữ liệu hoặc loại bài toán nhất định cũng như nghiên cứu về xu hướng học máy mới nhất, hãy kiểm tra [paperswithcode.com](http://paperswithcode.com/) và [sotabench.com](https://sotabench.com/).

## Hiển thị kết quả huấn luyện trên TensorBoard

Chúng ta đã theo dõi nhật ký huấn luyện fine-tuning của mô hình bằng cách sử dụng `TensorBoard` callback, hãy upload và kiểm tra chúng trên TensorBoard.dev.

In [ ]:
!tensorboard dev upload --logdir ./training_logs \
  --name "Fine-tuning EfficientNetB0 on all Food101 Data" \
  --description "Training results for fine-tuning EfficientNetB0 on Food101 Data with learning rate 0.0001" \
  --one_shot

In [ ]:
View experiment: https://tensorboard.dev/experiment/2KINdYxgSgW2bUg7dIvevw/

Xem [các training curve của mô hình trên TensorBoard.dev](https://tensorboard.dev/experiment/2KINdYxgSgW2bUg7dIvevw/), có vẻ như mô hình fine-tuning có tăng chất lượng nhưng bắt đầu quá khớp khi tiếp tục huấn luyện.

Xem các training curve trên TensorBoard.dev tại đây: https://tensorboard.dev/experiment/2KINdYxgSgW2bUg7dIvevw/

Để khắc phục điều này, ở các thử nghiệm tương lai, chúng ta có thể thử:
* Một vòng lặp khác của `EfficientNet` (ví dụ: `EfficientNetB4` thay vì `EfficientNetB0`).
* Bỏ đóng băng ít layer hơn của base model và huấn luyện chúng thay vì bỏ đóng băng toàn bộ base model trong một lần thực hiện.



In [ ]:
# Xem các thử nghiệm TensorBoard trước đây
!tensorboard dev list

In [ ]:
# Xóa các thử nghiệm TensorBoard trước đây
# !tensorboard dev delete --experiment_id YOUR_EXPERIMENT_ID

# Ví dụ
!tensorboard dev delete --experiment_id OAE6KXizQZKQxDiqI3cnUQ

## 🛠 Bài tập thực hành
> **Lưu ý:** Các bạn cần làm phần bài tập này để chuẩn bị cho phiên review lab.

1. Sử dụng các kỹ thuật đánh giá tương tự trên mô hình Food Vision quy mô lớn như chúng ta đã thực hiện trong notebook trước ([Transfer Learning Phần 3: Scaling up](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/06_transfer_learning_in_tensorflow_part_3_scaling_up.ipynb)). Cụ thể:
  * Confusion matrix giữa tất cả các dự đoán của mô hình và true label.
  * Biểu đồ hiển thị f1-score của mỗi lớp.
2. Trực quan hóa của mô hình đưa ra dự đoán trên các hình ảnh.
3. Huấn luyện lại mô hình (feature extraction và fine-tuning) mà chúng ta đã huấn luyện trong notebook này, ngoại trừ lần này sẽ sử dụng [`EfficientNetB4`](https://www.tensorflow.org/api_docs/python/tf/keras/applications/EfficientNetB4) làm base model thay vì `EfficientNetB0`. Bạn có thấy cải thiện về chất lượng không? Có tốn nhiều thời gian huấn luyện hơn không? Có bất kỳ đánh đổi nào cần xem xét không?
4. Kể tên một lợi ích quan trọng của mixed precision training, lợi ích này thế nào?

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
  plt.xticks(rotation=70, fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

In [ ]:
pred_classes = loaded_gs_model.predict(test_data)
y_labels = test_data.labels
make_confusion_matrix(y_true=y_labels,
                      y_pred=pred_classes,
                      classes=class_names,
                      figsize=(100, 100),
                      text_size=20,
                      norm=False,
                      savefig=True)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_labels, pred_classes))
class_f1_scores = {}
# Loop through classification report items
for k, v in classification_report_dict.items():
  if k == "accuracy": # stop once we get to accuracy key
    break
  else:
    # Append class names and f1-scores to new dictionary
    class_f1_scores[class_names[int(k)]] = v["f1-score"]
class_f1_scores

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 25))
scores = ax.barh(range(len(f1_scores)), f1_scores["f1-score"].values)
ax.set_yticks(range(len(f1_scores)))
ax.set_yticklabels(list(f1_scores["class_name"]))
ax.set_xlabel("f1-score")
ax.set_title("F1-Scores for 10 Different Classes")
ax.invert_yaxis(); # reverse the order

def autolabel(rects):
  for rect in rects:
    width = rect.get_width()
    ax.text(1.03*width, rect.get_y() + rect.get_height()/1.5,
            f"{width:.2f}",
            ha='center', va='bottom')

autolabel(scores)


In [47]:
#2. Trực quan hóa của mô hình đưa ra dự đoán
def predict_(img,model = model):
  img = tf.image.resize(img,(224,224))
  img_1 = tf.expand_dims(img,axis =0)
  kq = model.predict(img_1)
  kq_1 = np.argmax(kq)
  kq_2 = max(kq)
  kq_1 =  [kq_1]
  plt.imshow(img)
  plt.title(f'predict {kq_1} : {kq_2*100}%')
predict_(image_1)



SyntaxError: invalid decimal literal (<ipython-input-47-9863107719cd>, line 10)

In [2]:
#3.Huấn luyện lại mô hình sử dụng EfficientNetB4
input = tf.keras.layers.Input(shape=(224,224,3))
base_m = tf.keras.applications.EfficientNetB4(include_top=False)
base_m.trainable = True
x = base_m(input)
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
mode_b4 = tf.keras.Model(inputs, outputs)
model_b4.compile(loss="sparse_categorical_crossentropy", # Sử dụng sparse_categorical_crossentropy khi các nhãn *không phải* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
history_b4 = model_b4.fit(train_data,
                          epochs=100, # tinh chỉnh tối đa 100 epoch
                          steps_per_epoch=len(train_data),
                          validation_data=test_data,
                          validation_steps=int(0.15 * len(test_data)), # validation trong quá trình huấn luyện trên 15% dữ liệu kiểm tra
                          callbacks=[create_tensorboard_callback("training_logs", "efficientb0_101_classes_all_data_fine_tuning"), # theo dõi nhật ký huấn luyện của mô hình
                                      model_checkpoint, # chỉ lưu mô hình tốt nhất trong quá trình huấn luyện
                                      early_stopping, # ngừng mô hình sau X epoch mà không có cải thiện
                                      reduce_lr]) # giảm learning rate sau X epoch mà không có cải thiện


NameError: name 'tf' is not defined

##4. Kể tên một lợi ích quan trọng của mixed precision training :
Mixed precision training sử dụng tổ hợp của các kiểu dữ liệu precision đơn lẻ (float32) và half-preicison (float16) để tăng tốc độ huấn luyện mô hình (gấp 3 lần trên các GPU hiện đại).

## 📖 Tài liệu đọc thêm

* Đọc thêm về hoạch định learning rate và [learning rate scheduler callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/LearningRateScheduler). Đó là gì? Và nó hữu ích như thế nào đối với project này?
* Đọc thêm về TensorFlow data loaders ([cải thiện chất lượng data loading trong TensorFlow](https://www.tensorflow.org/guide/data_performance)). Chúng ta có bỏ lỡ điều gì không? Bạn nhớ được những phương pháp nào mỗi khi load dữ liệu trong TensorFlow? Gợi ý: xem lại phần tóm tắt ở cuối trang để hiểu hơn.
* Đọc thêm tài liệu về [TensorFlow mixed precision training](https://www.tensorflow.org/guide/mixed_precision). Chúng ta cần nhớ những điều quan trọng nào khi sử dụng mixed precision training?